In [12]:
import fitz
import spacy
from datetime import datetime

In [13]:
# Load NLP model
nlp = spacy.load('en_core_web_sm')

In [14]:
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [23]:
def extract_date(text):
    # Regex patterns to find dates
    date_pattern_english = r"On this date ([\w\s,]+), we the undersigned below:"
    date_pattern_indonesian = r"Pada hari ini tanggal ([\w\s,]+), pihak-pihak yang bertanda tangan di bawah ini:"

    # Search for dates in the first and last page
    match_english = re.search(date_pattern_english, text)
    match_indonesian = re.search(date_pattern_indonesian, text)

    date = None
    if match_english:
        date = match_english.group(1)
    elif match_indonesian:
        date = match_indonesian.group(1)
    
    return date

In [16]:
def extract_letter_number(text):
    # Regex patterns to find the letter number
    letter_number_pattern_english = r"Number:\s*([\w\-\/]+)"
    letter_number_pattern_indonesian = r"Nomor:\s*([\w\-\/]+)"

    # Search for letter number in the text
    match_english = re.search(letter_number_pattern_english, text)
    match_indonesian = re.search(letter_number_pattern_indonesian, text)

    letter_number = None
    if match_english:
        letter_number = match_english.group(1)
    elif match_indonesian:
        letter_number = match_indonesian.group(1)
    
    return letter_number

In [17]:
def extract_person_in_charge(text):
    return first_party, second_party

In [18]:
def extract_stakeholder_data(text):
    return stakeholder_data

In [19]:
def extract_supply_demand(text):
    return supply, demand

In [20]:
def extract_duration(text):
    return duration

In [21]:
def calculate_roi(supply, demand):
    return roi

In [22]:
def process_document(file_path):
    text = extract_text_from_pdf(file_path)
    
    date_of_agreement = extract_date(text)
    letter_number = extract_letter_number(text)
    first_party, second_party = extract_person_in_charge(text)
    stakeholder_data = extract_stakeholder_data(text)
    supply, demand = extract_supply_demand(text)
    duration = extract_duration(text)
    roi = calculate_roi(supply, demand)
    
    return {
        "Date of Agreement": date_of_agreement,
        "Letter Number": letter_number,
        "First Party Person in Charge": first_party,
        "Second Party Person in Charge": second_party,
        "Stakeholder Data": stakeholder_data,
        "Supply": supply,
        "Demand": demand,
        "Duration": duration,
        "ROI": roi
    }